In [ ]:
import holoviews as hv
import panel as pn

from py_eddy_tracker.data import get_remote_demo_sample
from py_eddy_tracker.generic import flatten_line_matrix
from py_eddy_tracker.poly import create_vertice
from py_eddy_tracker.observations.network import NetworkObservations
from numpy import where, datetime64, timedelta64

In [ ]:
import pylook
from os.path import dirname
COAST = pylook.coast.CoastFile(f"{dirname(pylook.__file__)}/gshhs_backup/binned_GSHHS_l.nc")

In [ ]:
hv.extension('bokeh', logo=False)
pn.extension()

In [ ]:
a = NetworkObservations.load_file(
    get_remote_demo_sample("eddies_med_adt_allsat_dt2018_err70_filt500_order1/Anticyclonic_network.nc")
)

In [ ]:
def visu(self, intern=True):
    xname, yname = self.intern(intern)
    self[xname][:] = (self[xname] + 180) % 360 -180
    slider = pn.widgets.DiscretePlayer(name="date", options=list(range(*self.period)), value=self.period[0])

    def coast(x_range, y_range):
        if x_range is None:
            x_range = 0,0
            y_range = 0,0
        area = dict()
        area['llcrnrlon'], area['urcrnrlon'] = x_range
        area['llcrnrlat'], area['urcrnrlat'] = max(y_range[0], -90), min(y_range[1], 90)
        return hv.Path([COAST.lines(**area).get_paths()[0].vertices]).options(aspect='equal', color='gray')

    @pn.depends(slider)
    def update(value):
        i = where(self.time == value)[0]
        x, y = flatten_line_matrix(self[xname][i]), flatten_line_matrix(self[yname][i])
        return hv.Path([create_vertice(x, y)]).options(line_width=2)

    @pn.depends(slider)
    def plot_date(value):
        return pn.pane.HTML(f"<h1>date = {datetime64('1950-01-01') + timedelta64(value)}</h1>")

    eddies = hv.DynamicMap(update)
    coast_ = hv.DynamicMap(coast, streams=[hv.streams.RangeXY(source=eddies, x_range=(0,30), y_range=(30,45))])
    return pn.Column(pn.Row(slider, plot_date), (eddies * coast_).opts(width=900, height=500))


visu(a)